# Brije: Cognitive Action Detection - Full Pipeline (Google Colab)

Complete pipeline for training cognitive action probes on Gemma 3 4B.

**This notebook will:**
1. ✅ Clone the Brije repository
2. ✅ Install all dependencies
3. ✅ Capture activations from Gemma 3 4B (~2-3 hours)
4. ✅ Train probes on captured activations (~20-30 minutes)
5. ✅ Test and evaluate performance
6. ✅ Download trained models to Google Drive

**Requirements:**
- Google Colab with A100 GPU (40GB VRAM recommended)
- Runtime: ~3-4 hours total

**Dataset:** 31,500 cognitive action examples across 45 actions

## 1️⃣ Check GPU and Setup Runtime

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print("\n" + "="*60)
print("GPU INFORMATION")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  WARNING: No GPU detected! This will be very slow on CPU.")
print("="*60)

## 2️⃣ Clone Repository and Install Dependencies

In [ ]:
import os
import sys

# Clone the repository
repo_url = "https://github.com/ChuloIva/brije.git"
repo_name = "brije"

if not os.path.exists(repo_name):
    print("📥 Cloning Brije repository...")
    !git clone {repo_url}
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists")
    print("🔄 Pulling latest changes...")
    !cd {repo_name} && git pull

# Change to repo directory
os.chdir(repo_name)
print(f"\n📁 Current directory: {os.getcwd()}")

In [ ]:
# Install dependencies
print("📦 Installing dependencies...\n")
!pip install -q torch transformers h5py scikit-learn tqdm matplotlib seaborn

# Install nnsight from third_party
print("\n📦 Installing nnsight...")
!pip install -q -e third_party/nnsight

print("\n✅ All dependencies installed!")

## 3️⃣ Mount Google Drive (for saving outputs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directories in Google Drive for outputs
drive_output_dir = '/content/drive/MyDrive/brije_outputs'
os.makedirs(drive_output_dir, exist_ok=True)
os.makedirs(f"{drive_output_dir}/activations", exist_ok=True)
os.makedirs(f"{drive_output_dir}/probes", exist_ok=True)

print(f"✅ Outputs will be saved to: {drive_output_dir}")

## 4️⃣ Verify Dataset

In [ ]:
# Check if dataset exists
import glob

dataset_path = "third_party/datagen/generated_data"
datasets = glob.glob(f"{dataset_path}/*.jsonl")

print("="*60)
print("AVAILABLE DATASETS")
print("="*60)
for ds in datasets:
    size = os.path.getsize(ds) / 1e6
    print(f"  {os.path.basename(ds)} ({size:.2f} MB)")
print("="*60)

# Use the stratified combined dataset (31.5k examples)
dataset_file = None
for ds in datasets:
    if 'stratified_combined' in ds or '31500' in ds:
        dataset_file = ds
        break

if not dataset_file:
    # Use any available dataset
    dataset_file = datasets[0] if datasets else None

if dataset_file:
    print(f"\n✅ Using dataset: {os.path.basename(dataset_file)}")
else:
    print("\n⚠️  No dataset found! You may need to generate data first.")
    print("See: third_party/datagen/README.md")

## 5️⃣ Configure Pipeline Parameters

In [ ]:
# Configuration
CONFIG = {
    'model': 'google/gemma-3-4b-it',
    'dataset': dataset_file,
    'layers_to_capture': [7, 14, 21, 27],  # Evenly spaced layers
    'target_layer': 27,  # Layer to train probe on
    'probe_type': 'multihead',  # 'linear' or 'multihead'
    'batch_size': 32,
    'epochs': 20,
    'learning_rate': 0.001,
    'device': 'auto',
    'max_examples': None,  # None = use all examples, or set a number for quick test
}

print("="*60)
print("PIPELINE CONFIGURATION")
print("="*60)
for key, value in CONFIG.items():
    print(f"  {key:20s}: {value}")
print("="*60)

# For quick testing (uncomment to test with smaller dataset)
# CONFIG['max_examples'] = 1000
# CONFIG['epochs'] = 5
# CONFIG['layers_to_capture'] = [27]  # Just one layer for quick test

## 6️⃣ Step 1: Capture Activations (~2-3 hours)

This extracts hidden states from Gemma 3 4B at specified layers.

**⏰ Expected time:** ~2-3 hours for full dataset (31.5k examples)

**💾 Memory:** ~12-16 GB VRAM peak usage

In [ ]:
import time

print("\n" + "="*60)
print("STEP 1: CAPTURING ACTIVATIONS")
print("="*60)
print(f"Model: {CONFIG['model']}")
print(f"Layers: {CONFIG['layers_to_capture']}")
print(f"Dataset: {os.path.basename(CONFIG['dataset'])}")
print("\n⏰ This will take 2-3 hours. Progress will be displayed below.")
print("💡 Activations are cached, so re-running is instant!\n")

start_time = time.time()

# Build command
cmd = [
    'python', 'src/probes/capture_activations.py',
    '--dataset', CONFIG['dataset'],
    '--output-dir', 'data/activations',
    '--model', CONFIG['model'],
    '--layers', *[str(l) for l in CONFIG['layers_to_capture']],
    '--device', CONFIG['device'],
    '--format', 'hdf5'
]

if CONFIG['max_examples']:
    cmd.extend(['--max-examples', str(CONFIG['max_examples'])])

# Run capture
!{' '.join(cmd)}

elapsed = time.time() - start_time
print(f"\n✅ Activation capture completed in {elapsed/3600:.2f} hours ({elapsed/60:.1f} minutes)")

# Copy to Google Drive for backup
print("\n📥 Backing up activations to Google Drive...")
!cp -r data/activations/* {drive_output_dir}/activations/
print("✅ Backup complete!")

## 7️⃣ Step 2: Train Probe (~20-30 minutes)

Train a probe to detect cognitive actions from the captured activations.

**⏰ Expected time:** 15-30 minutes

**🎯 Expected accuracy:** 70-85% (45-way classification)

In [ ]:
print("\n" + "="*60)
print("STEP 2: TRAINING PROBE")
print("="*60)
print(f"Layer: {CONFIG['target_layer']}")
print(f"Probe type: {CONFIG['probe_type']}")
print(f"Epochs: {CONFIG['epochs']}")
print(f"Batch size: {CONFIG['batch_size']}")
print("\n⏰ This will take 15-30 minutes.\n")

start_time = time.time()

# Build command
activation_file = f"data/activations/layer_{CONFIG['target_layer']}_activations.h5"

cmd = [
    'python', 'src/probes/train_probes.py',
    '--activations', activation_file,
    '--output-dir', 'data/probes',
    '--model-type', CONFIG['probe_type'],
    '--batch-size', str(CONFIG['batch_size']),
    '--epochs', str(CONFIG['epochs']),
    '--lr', str(CONFIG['learning_rate']),
    '--device', CONFIG['device']
]

# Run training
!{' '.join(cmd)}

elapsed = time.time() - start_time
print(f"\n✅ Probe training completed in {elapsed/60:.1f} minutes")

# Copy to Google Drive
print("\n📥 Backing up trained probe to Google Drive...")
!cp -r data/probes/* {drive_output_dir}/probes/
print("✅ Backup complete!")

## 8️⃣ View Training Results

In [ ]:
import json

# Load test metrics
metrics_file = 'data/probes/test_metrics.json'
if os.path.exists(metrics_file):
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    print("="*60)
    print("TEST PERFORMANCE METRICS")
    print("="*60)
    print(f"Test Accuracy:     {metrics['test_accuracy']:.2%}")
    print(f"Macro F1 Score:    {metrics['macro_f1']:.4f}")
    print(f"Top-3 Accuracy:    {metrics['top3_accuracy']:.2%}")
    print(f"Top-5 Accuracy:    {metrics['top5_accuracy']:.2%}")
    print("="*60)
    
    # Show best and worst classes
    if 'per_class_f1' in metrics:
        per_class = metrics['per_class_f1']
        sorted_classes = sorted(per_class.items(), key=lambda x: x[1], reverse=True)
        
        print("\nBest performing actions:")
        for action, f1 in sorted_classes[:5]:
            print(f"  {action:30s} F1: {f1:.3f}")
        
        print("\nWorst performing actions:")
        for action, f1 in sorted_classes[-5:]:
            print(f"  {action:30s} F1: {f1:.3f}")
else:
    print("⚠️  Metrics file not found")

## 9️⃣ Step 3: Test Probe Inference

Test the trained probe on sample texts.

In [ ]:
# Test on sample texts
test_texts = [
    "After receiving feedback, she began reconsidering her initial approach to the problem.",
    "He was analyzing the data to find patterns and correlations between variables.",
    "They started generating creative ideas for solving the design challenge.",
    "She was evaluating different strategies to determine the most effective one.",
    "He tried to recall the specific details from the previous meeting."
]

print("="*60)
print("INFERENCE EXAMPLES")
print("="*60)

for text in test_texts:
    print(f"\n📝 Text: {text}")
    
    cmd = [
        'python', 'src/probes/probe_inference.py',
        '--probe', 'data/probes/best_probe.pth',
        '--model', CONFIG['model'],
        '--layer', str(CONFIG['target_layer']),
        '--text', text,
        '--top-k', '3'
    ]
    
    !{' '.join(cmd)}
    print("")

## 🔟 Visualize Training History (Optional)

In [ ]:
import matplotlib.pyplot as plt
import json

history_file = 'data/probes/training_history.json'
if os.path.exists(history_file):
    with open(history_file, 'r') as f:
        history = json.load(f)
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Plot loss
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss')
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot accuracy
    axes[1].plot(epochs, history['train_acc'], 'b-', label='Train Accuracy')
    axes[1].plot(epochs, history['val_acc'], 'r-', label='Val Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('data/probes/training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Training curves saved to: data/probes/training_curves.png")
else:
    print("⚠️  Training history not found")

## 1️⃣1️⃣ Download Trained Models

Download the trained probe and metrics for local use.

In [ ]:
from google.colab import files
import shutil

# Create a zip file with all outputs
output_zip = 'brije_trained_probe.zip'

print("📦 Creating download package...")
!cd data && zip -r ../{output_zip} probes/

print(f"\n✅ Package created: {output_zip}")
print(f"Size: {os.path.getsize(output_zip) / 1e6:.2f} MB")
print("\n📥 Starting download...")

files.download(output_zip)

print("\n✅ Download complete!")
print("\nPackage contains:")
print("  • best_probe.pth - Best performing model")
print("  • final_probe.pth - Final epoch model")
print("  • test_metrics.json - Performance metrics")
print("  • training_history.json - Training curves")
print("  • confusion_matrix.png - Confusion matrix plot")

## 1️⃣2️⃣ Summary and Next Steps

In [ ]:
print("="*60)
print("🎉 PIPELINE COMPLETE!")
print("="*60)
print("\n✅ What was accomplished:")
print("  1. Captured activations from Gemma 3 4B")
print("  2. Trained probe for cognitive action detection")
print("  3. Evaluated on test set")
print("  4. Saved all outputs to Google Drive")
print("\n📂 Outputs saved to:")
print(f"  • Local: {os.getcwd()}/data/")
print(f"  • Google Drive: {drive_output_dir}")
print("\n🚀 Next Steps:")
print("  1. Download the trained probe (see cell above)")
print("  2. Use probe in liminal_backrooms GUI")
print("  3. Test on your own texts")
print("  4. Train on different layers for comparison")
print("\n📚 Documentation:")
print("  • README.md - Full documentation")
print("  • QUICK_REFERENCE.md - Command cheat sheet")
print("="*60)